# Playing Atari with Deep Reinforcement Learning Review
  
  
  
강화학습 분야에서 유명한 논문인 'Playing Atari with Deep Reinforcement Learning'을 리뷰해보겠습니다. 전체적인 흐름은 논문을 따라가겠습니다.

## Abstract

1. Reinforcement Learning ( 이후 RL )을 사용해 high-dimensional sensory input을 바탕으로 control policies를 성공적으로 학습한 첫 번째 deep learning model을 만들었다.  
  
  
2. 모델은 Q-learning의 변형을 학습시틴 cnn 모델이며, input으로는 raw pixels, output은 미래 reward를 계산하는 value function이다.  
  
  
3. 이 방식을 learning algorithm이나 architecture의 변형 없이, Atari game 중 7개에 적용했고, 이 중 6개는 이전의 모델들 보다 더 뛰어난 성능을 얻었고, 이들 중 3개는 사람보다 더 높은 성능을 보였다.

## Introduction  

1. 대부분 성공적인 RL application들은 linear value functions 이나 policy representations과 결합한 hand-crafted feature에 의존했다. 즉, 시스템의 성능이 feature의 질에 의해 결정됐다.  
  
  
2. 그러나 최근 deep learning의 발달로 raw sensory data에서 높은 수준의 feature들을 추출하는게 가능해졌다. 그래서 이 방식이 RL에 도움이 될 것이라고 판단했다.  
  
  
3. 그러나 딥러닝을 강화학습에 적용하기에는 __몇 가지 문제점들__이 발생했다.  
  
  1) 딥러닝 모델의 경우, 많은 양의 hand-labelled training data를 필요로 한다. 하지만 강화학습은 __sparse, noisy, delayed한 scalar reward signal__로 학습을 해야한다.  
    
  2) 딥러닝 알고리즘의 샘플 데이터는 서로 독립임을 가정하나, 강화학습의 경우 시계열적, 즉 __sequence 간에 상관성이 굉장히 높다.__  
    
  3) 데이터의 분포가 고정된 딥러닝과는 달리, 강화학습은 데이터 샘플의 분포가 알고리즘이 새로운 행동을 학습할 때마다 __분포가 변한다.__  
      
      
4. 이 논문에서는 cnn이 복잡한 강화학습 환경에서 raw video data를 통해 성공적으로 control policies들을 학습함으로써 문제점을 극복할 수 있다고 말한다.  
  - __Q-learning algorithm의 변형 + sgd 방식의 weight update + replay mechanism__

## Background  
  
1. atari에 적용된 방법은 아래와 같다.  
  
  - 각 time step 마다 action을 선택한다.  
  - 이 action은 emulator로 전달되고 state와 score를 결정한다.  
  - reward를 바꾼다. (최종 결과는 수천회의 반복 이후 얻을 수 있음)  
      
      
2. 그러나 agent가 오로지 현재의 screen만 보고 학습하기에는 불가능하다. 그래서 $s_t$를 고려하며 이 __sequence는 유한하다__고 가정한다. 그래서 MDP 방식을 적용할 수 있고, 이를 standard reinforcement learning에 적용할 수 있게 된다.  
    
    
3. __agent의 목표는 emulator와 상호작용하여 future reward를 최대화하는 방식의 action을 선택하는 것__이다. 이를 구하기 위해서 아래와 같이 정의해야한다.  
  
  - $R_t = \sum_{t' = t}^T (\gamma^{t'-t}r_t) $  
  - $Q^*(s,a) = max_\pi E[R_t|s_t = s, a_t = a, \pi]$  
      
      
4. 이 때, $Q^*(s,a)$는 bellman equation의 성질에 근거한다. 즉, 아래와 같이 식을 변형할 수 있게 된다.  
      
    -  $Q^*(s,a) = E_{s'~ \epsilon}[r + \gamma max_{a'} Q^*(s',a')|s,a]$    
    
      
5. 이 식을 바탕으로 iteration을 수렴할 때까지 반복하면, 최적의 action-value function인 $Q^*$를 찾을 수 있다. 그러나 action-value function은 각 sequnce마다 측정해서 이 방식을 적용할 수 없으므로 __$Q(s, a; θ) ≈ Q∗(s, a)$의 function approximator를 사용__하기로 한다.   
    
    
6. 여기서는 Q-network라는 $\theta$를 weight로 두는 neural network function approximator를 사용한다. 이 network는 아래의 loss function을 최소화하며 학습한다.  
  
  - $L_i(\theta_i) = E_{s,a~\rho(.)}[(y_i - Q(s,a;\theta_i))^2]$
  - 이 때, $\rho(s,a)$는 sequence가 s고, action이 a인 behaviour distribution을 말합니다.  
    
    
7. 이러한 가정을 바탕으로 구해지는 gradient는 아래와 같습니다.  
  
  - $\nabla_\theta L_i(\theta_i) = E_{s,a~\rho(.);s'~\epsilon}[(r + \gamma max_{a'} Q^*(s',a';\theta_{i-1}) - Q(s,a;\theta_i)\nabla_\theta Q(s,a;\theta_i)]$  
        
      이를 모든 time step에 적용하면 Q-learning 알고리즘과 유사한 알고리즘을 얻을 것이다.  
  
      
8. 위에서 도출한 알고리즘은 __model-free__ 알고리즘이며, __off-policy__이다. 또한 __$\epsilon$ - greedy strategy__를 사용했다.


### cf  
  
### off - policy  :
  
   학습하는 policy와 행동하는 policy가 반드시 같지 않아도 학습이 가능한 알고리즘.   
      
### $\epsilon$ - greedy strategy :  

   각 단계에서 최선의 선택을 한 것이 최종적으로 최선의 선택이기를 바라는 알고리즘. $\epsilon$ 만큼은 random하게, 1-$\epsilon$은 최선의 선택을 하게 함으로써 탐험을 충분히 할 수 있게끔 만든 알고리즘

## Related Work  
  
  1. 이전에 가장 성공했던 RL모델은 TD - gammon이었다. 이 모델 역시 model-free RL 알고리즘을 사용했고, approximated value function을 사용했다. 그러나 이 모델은 backgammon 게임 한정 알고리즘이었다. (주사위의 확률이 탐험과 value function의 smooth를 도와줌) 게다가 모델이 학습할 때, linear function approximator를 사용해야 더 수렴할 것이라는 보장이 된다.
      
      
  2. 그러나 최근에는 딥러닝과 강화학습의 결합을 고려하게 됐다. 이로 인해 고정된 policy에 nonlinear function approximator를 적합하거나 control policy에 linear function approximator를 적합할 수 있게 됐지만 완벽하게 nonlinear control은 불가능 했다.   
      
      
  3. 논문과 유사한 과거의 모델은 NFQ가 있다. NFQ는 Q-network의 파라미터를 갱신하기 위해 RPROP 알고리즘을 사용했고, loss function을 최적화시켰다.  
      
      
  4. 이전의 NFQ 모델에서 더 발전시켜, 논문의 모델은 bgd가 아닌 sgd를 사용해, 연산량을 줄였다. 또한 NFQ같은 경우, 일종의 전처리를 통해 학습시켰으나 논문의 모델은 처음부터 끝까지 철저히 강화학습을 시켰다.

## Deep Reinforcement Learning  
  
  1. 대부분의 성공적인 접근법은 raw input을 학습하고, sgd 방식으로 lightweight를 업데이트 하는 것이다. 이를 바탕으로 강화학습을 진행하기로 했다. 최종 목표는 __RL 알고리즘을 sgd를 사용해 효율적으로 학습 데이터를 처리하고 RGB 이미지를 받아드릴 수 있는 Deep Neural Network과 연결하는 것__이다.  
    
  2. 위 목표를 위해 논문에서는 __experience replay 기술__을 사용했으며, 이를 알고리즘에 적용시켰다. 그리고 이 알고리즘이 Deep Q - learning이다. 알고리즘은 아래의 그림과 같다.  
    
![dqn](desktop/dqn.png)  
  
- 각 time-step별로 $e_t = (s_t, a_t, r_t, s_{t=1})$를 구하고, 이를 데이터셋인 $\mathcal{D} = e_1,...,e_N$의 형태로 __replay memory__로 저장한다.  
  
- 저장해둔 $\mathcal{D}$에서 무작위로 minibatch의 sample을 바탕으로 update한다.  
  
- experience replay과정을 거친 구, agent는 action을 $\epsilon$ - greedy policy에 따라 선택한다.
      
        
3. 이 방식은 아래와 같은 장점이 있다.  
  
  - 데이터 효율성을 높여, 각 step of experience에서 더 많은 weight update의 잠재성을 가진다.  
    
  - sample들 사이에 있던 높은 상관성을 randomizing을 통해 없앨 수 있으며 update의 variance를 줄여준다.  
    
  - experience replay를 함으로써, behavior distribution은 이전 states에 비해 average된다. 이 때 염두해야 할 점은 off - policy로 학습해야 한다는 점이다.  
      
      
4. 단점은 memory size가 N이어서 계속해서 덧씌워야 한다는 점과 중요한 변화들을 구분하지 못한다는 점이다.

### Deep Reinforcement learning  - 1
  
1. 기본적으로 데이터의 형태는 128개 컬러의 210*160 pixel image가 들어온다.  
    
    
2. 이 input을 gray-scaling하고 110 * 84 형태로 줄이고 마지막으로 84 * 84로 crop해서 최종 input 데이터로 전처리한다.  
    
    
3. 논문의 실험에서는 마지막 4개의 프레임에 대해서만 모아서 전처리를 진행하고, 이를 Q-function의 input으로 처리했다.  
  
    
4. 또한 history만을 input으로, output은 각 action별 예측된 Q-value를 구하는 방법을 모델에 적용했다. (한번의 forward pass로 output을 구할 수 있어서 연산량이 줄어듬)  
  
  
5. 이 모델을 DQN (Deep Q-Networks)라고 명칭을 붙였다.   
  
![dqn2](desktop/dqn2.png)

## Experiments  
  
    
1. 이 DQN 모델에게 atari 게임 7개를 학습시켰다. 이 때 게임에 대한 사전 정보 등은 제공하지 않았고, 별다른 조정 또한 하지 않았다. 대신 reward의 구조만을 손봤다. __positive reward는 1, negative reward는 -1, 변하지 않는다면 0으로 설정했다.__  
  
  
2. 실험에서는 RMSProp 알고리즘을 사용했고, minibatch의 size는 32로 설정했다. $\epsilon$ - greedy에서 $\epsilon$는 1부터 0.1로 백만번 진행하면서 점차 줄어들게 했고, 그 이후는 0.1로 고정했다.  
  
  
3. 특히, 중요한 점은 학습을 진행할 때, __frame-skipping technique__를 사용했단 점이다. 이 기술을 적용함으로써 연산량이 확연히 줄게된다.

### Experiments - 1  
  
    
1. average total reward metric에 대한 그래프는 굉장히 지저분한 경향이 있다. 왜냐하면 policy의 weight가 조금만 변해도 state의 분산이 크게 변하기 때문이다.  
  
    
2. policy's estimated action-value function Q는 다른 지표로 사용되는데 이 지표가 위보다 훨씬 안정적으로 증가하기 때문에 사용하기 적절하다.  
  
![graph](desktop/graph.png)  
  
    
### Experiments - 2  (생략)

### Experiments - 3  
  
    
1. DQN 모델을 이전의 다른 모델들의 성능과 비교해보기로 한다.  
  
  - 기본적으로 sarsa, contingency 등의 이전의 RL 모델들보다는 훨씬 성능이 향상됐다. 뿐만 아니라 3개의 게임에서는 사람보다 더 높은 점수를 달성했다.  
    
  - HNeat Best와 HNeat Pixel과 비교해봐도 S.Invaders를 제외한 모든 게임에서 outperform한 성능을 나타냈다.  
    
![table](desktop/table.png)

## Conclusion  
  
1. 이 논문은 RL에 적용한 Deep learning model을 소개했으며 input으로서 raw pixel을 사용해 Atari 2600 컴퓨터 게임을 성공적으로 학습한 모습을 설명했다.  
    
    
2. 또한 __experience replay memory__를 활용해 __stochastic minibatch update__를 하는 __Q-learning__의 변형을 선보였다.  
  
    
3. 그 결과, 어떠한 조정없이 7개의 게임 중 6개에서 성공적인 결과를 보여줄 수 있었다.

참고 자료  
  
https://newsight.tistory.com/250  

https://dongminlee.tistory.com/3  

https://mangkyu.tistory.com/60